The following are several read alignment algorithm

In [3]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

Naive match with max_mis-match

In [6]:
def naive_mm(p, t,max_mismatch):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        mismatch = 0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                mismatch += 1
                if(mismatch>max_mismatch):
                    match = False
                    break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences


In [8]:
virus_genome = readGenome('lambda_virus.fa')
occurrences = naive_mm('TTCAAGCC', virus_genome,2)
print "There are ",len(occurrences), "matches."

There are  191 matches.


Boyer-Moore algorithm

In [9]:
from bm_preproc import BoyerMoore

In [10]:
def boyer_moore_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    num_character_comparisons = 0
    num_alignments = 0
    while i < len(t) - len(p) + 1:
        num_alignments += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            num_character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences,num_alignments, num_character_comparisons

In [11]:
human_genome = readGenome('chr1.GRCh38.excerpt.fasta')
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
uppercase_alphabet = 'ATCG '
p_bm = BoyerMoore(p, uppercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, human_genome)
print(occurrences, num_alignments, num_character_comparisons)

([56922], 127974, 165191)


Alignment with Kmer Index

In [12]:
from kmer_index import Index

In [19]:
def approximate_match(p, t, n,k):
    all_matches = set()
    for i in range(len(p)/k):
        start = i*k
        end = min((i+1)*k, len(p))
        matches = myindex.query(p[start:end])
        #print len(matches)
        # Extend matching segments to see if whole p matches
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches)

In [22]:
human_genome = readGenome('chr1.GRCh38.excerpt.fasta')
p ='GGCGCGGTGGCTCACGCCTGTAAT'
k = 8   #kmer:k=8
myindex = Index(human_genome,k)
matches = approximate_match(p, human_genome, 2, k)
print "match index:",matches

match index: [84641, 160162, 635931, 747359, 273669, 147558, 364263, 681737, 717706, 465647, 429299, 657496, 160729, 56922, 724927, 191452, 262042, 551134, 421221]


Approximate matching with DP

In [23]:
def distance_approximate_occurance(x, y):#y is t and p is x
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0] * (len(y)+1))
        
    # Initialize first column
    for i in range(1, len(x)+1):
        D[i][0] = i

    # Initialize first row
    for j in range(1,len(y)+1):
        D[0][j] = 0
        
    # Fill rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    return min(D[-1])  # return the min value of last low

In [25]:
p ='GCTGATCGATCGTACG'
distance_approximate_occurance(p,human_genome)

3